In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
API_KEY = user_secrets.get_secret("avalai_api")

In [ ]:
%pip install -q openai ragas langchain_openai nltk rouge 

Json of AgenticRAG_openai_openai


In [ ]:
!gdown --id "17XpMWVjsHobPSQkqdCD2inql4HN9d92C"

In [ ]:
import os
# from datasets import Dataset
from openai import OpenAI
from ragas import evaluate
from ragas.metrics import (
    Faithfulness, 
    AnswerRelevancy, 
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge

# --- RAGAS Evaluation ---

# AvalAI credentials
# API_KEY = "aa-"
BASE_URL = "https://api.avalai.ir/v1"

# Initialize AvalAI LLM
def get_eval_llm():
    """Initializes and returns the Language Model for evaluation."""
    return LangchainLLMWrapper(
        ChatOpenAI(
            base_url=BASE_URL,
            model="gpt-4o-mini",
            api_key=API_KEY
        )
    )

client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

def create_embedding(text):
    """Creates an embedding for the given text using the specified model."""
    resp = client.embeddings.create(model="text-embedding-3-large", input=text)
    return resp.data[0].embedding

class CustomEmbeddingModel:
    """A custom embedding model wrapper for RAGAS."""
    def embed_query(self, text: str):
        """Embeds a single query."""
        return create_embedding(text)
    def embed_documents(self, texts: list[str]):
        """Embeds a list of documents."""
        return [create_embedding(t) for t in texts]

def get_eval_emb():
    """Initializes and returns the embedding model for evaluation."""
    return LangchainEmbeddingsWrapper(CustomEmbeddingModel())

def evaluate_ragas(user_input, retrieved_contexts, generated_answer, reference_answer=None):
    """
    Evaluates the RAG pipeline using the Ragas framework.
    The reference_answer is optional. If not provided, metrics requiring it will not be run.

    Args:
        user_input (str): The user's query.
        retrieved_contexts (list[str]): The list of retrieved document contexts.
        generated_answer (str): The answer generated by the RAG model.
        reference_answer (str, optional): The ground truth or reference answer. Defaults to None.

    Returns:
        dict: A dictionary containing the evaluation scores.
    """
    data = {
        "question": [user_input],
        "contexts": [retrieved_contexts],
        "answer": [generated_answer],
    }
    if reference_answer:
        data["ground_truth"] = [reference_answer]

    ds = Dataset.from_dict(data)

    llm = get_eval_llm()
    emb = get_eval_emb()

    metrics = [
        Faithfulness(llm=llm),
        AnswerRelevancy(llm=llm, embeddings=emb),
    ]

    result = evaluate(dataset=ds, metrics=metrics, llm=llm, embeddings=emb, show_progress=False)
    return result.to_pandas().iloc[0].to_dict()

# --- BLEU Score Evaluation ---

def evaluate_bleu(reference, candidate):
    """
    Calculates the BLEU score for a candidate sentence against a reference.

    Args:
        reference (list[str]): A list of reference sentences.
        candidate (str): The candidate sentence to evaluate.

    Returns:
        float: The BLEU score.
    """
    reference_tokens = [ref.split() for ref in reference]
    candidate_tokens = candidate.split()
    chencherry = SmoothingFunction()
    return sentence_bleu(reference_tokens, candidate_tokens, smoothing_function=chencherry.method1)

# --- ROUGE Score Evaluation ---

def evaluate_rouge(reference, candidate):
    """
    Calculates the ROUGE score for a candidate sentence against a reference.

    Args:
        reference (str): The reference sentence.
        candidate (str): The candidate sentence to evaluate.

    Returns:
        dict: A dictionary containing ROUGE scores (rouge-1, rouge-2, rouge-l).
    """
    rouge = Rouge()
    scores = rouge.get_scores(candidate, reference)
    return scores[0]

In [ ]:
# evaluate_dataset.py
import json
import pathlib
from statistics import mean

import pandas as pd

# ---- import your metrics helpers ----
# from my_metrics import (
#     evaluate_ragas,
#     evaluate_bleu,
#     evaluate_rouge,
# )

# ---------- utility helpers ----------
def load_jsonl(path: str) -> list[dict]:
    """Reads the dataset (one JSON object per line)."""
    with open(path, "r", encoding="utf‑8") as f:
        return [json.loads(line) for line in f]

def read_context(path: str) -> str:
    """Reads the document that was retrieved for the question."""
    with open(path, "r", encoding="utf‑8") as f:
        return f.read()

# ------------- main loop -------------
def evaluate_dataset(json_path: str) -> pd.DataFrame:
    rows = load_jsonl(json_path)
    results = []

    for i, row in enumerate(rows, start=1):
        q          = row["question"]
        gen_ans    = row["answer"]                       # model output
        ref_ans    = row.get("reference_answer")         # may be None
        ctx_text   = read_context(row["file"])

        # -- RAGAS (faithfulness & answer‑relevancy) --
        ragas_scores = evaluate_ragas(
            user_input=q,
            retrieved_contexts=[ctx_text],
            generated_answer=gen_ans,
            reference_answer=ref_ans,                    # OK if None
        )

        # -- BLEU & ROUGE (only if ground truth provided) --
        if ref_ans:
            bleu  = evaluate_bleu([ref_ans], gen_ans)
            rouge = evaluate_rouge(ref_ans, gen_ans)
        else:
            bleu, rouge = None, {}

        # ---- store everything ----
        results.append(
            {
                "id": i,
                "question": q,
                "file": row["file"],
                "generated_answer": gen_ans,
                "reference_answer": ref_ans,
                "faithfulness": ragas_scores["faithfulness"],
                "answer_relevancy": ragas_scores["answer_relevancy"],
                "bleu": bleu,
                **{f"rouge_{k}": v["f"] for k, v in rouge.items()} if rouge else {},
            }
        )

    df = pd.DataFrame(results)
    return df


if __name__ == "__main__":
    DATA_PATH = "data/eval_set.jsonl"        # <‑‑ change to your file
    OUT_PATH  = "data/eval_results.csv"

    df = evaluate_dataset(DATA_PATH)
    print("\n=== Per‑row scores ===")
    print(df.to_markdown(index=False))

    # ---- corpus‑level means ----
    numeric_cols = ["faithfulness", "answer_relevancy", "bleu",
                    "rouge_1", "rouge_2", "rouge_l"]
    corpus_avgs = {c: mean(df[c].dropna()) for c in numeric_cols if c in df}
    print("\n=== Averages ===")
    for k, v in corpus_avgs.items():
        print(f"{k:>16}: {v:.4f}")

    # ---- save if you like ----
    pathlib.Path(OUT_PATH).parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(OUT_PATH, index=False)
    print(f"\nDetailed results written to {OUT_PATH}")
